In [29]:
import math
import copy
import time

def forward_selection(data, num_features, num_instances):
    best_so_far_accuracy = 0.0
    #initialize an empty set
    best_subset = [] #initialize the best subset so far for a designated number of features
    best_total = [] #the best set among all of different numbers of features.
    for i in range(num_features): #start from 0 feature
        #flags for checking if the new feature is added to total or subset
        add_total = -1
        best_subset_accuracy = 0.0
        print("On the " , i , "th level of the search tree")

        for j in range(1, num_features + 1):
            if j not in best_subset:
                cur_subset = copy.deepcopy(best_subset)
                cur_subset.append(j)
                
                accuracy = leave_one_out_cross_validation(data, cur_subset, num_instances)
                print("\tUsing feature(s) ", cur_subset, " accuracy is ", accuracy, "%")
                if accuracy > best_so_far_accuracy:
                    best_so_far_accuracy = accuracy
                    add_total = j
                if accuracy > best_subset_accuracy:
                    best_subset_accuracy = accuracy
                    add_subset = j
        
        if add_total != -1:
            best_subset.append(add_total)
            best_total.append(add_total)
            print("\n\nFeature set ", best_subset, " was best, accuracy is ", best_so_far_accuracy, "%\n\n")
        else:
            print("\n\n(Warning, Accuracy has decreases! Continuing search in case of local maxima)")
            best_subset.append(add_subset)
            print("Feature set ", best_subset, " was best, accuracy is ", best_subset_accuracy, "%\n\n")
    #Finished looping all numbers of features
    print("Finished search!! The best feature subset is ", best_total, " which has an accuracy of ", best_so_far_accuracy, "%")
    


def backward_elimination(data, num_features, num_instances):
    best_so_far_accuracy = 0.0
    #initialize a full subset
    best_subset = [i+1 for i in range(num_features)] #initialize the best subset so far for a designated number of features
    best_total = [i+1 for i in range(num_features)] #the best set among all of different numbers of features.
    for i in range(num_features):
         #flags for checking if the new feature is added to total or subset
        remove_total = -1
        best_subset_accuracy = 0.0
        print("On the " , i , "th level of the search tree")
        
        for j in range(1, num_features + 1):
            if j in best_subset:
                cur_subset = copy.deepcopy(best_subset)
                cur_subset.remove(j)
                
                accuracy = leave_one_out_cross_validation(data, cur_subset, num_instances)
                print("\tUsing feature(s) ", cur_subset, " accuracy is ", accuracy, "%")
                if accuracy > best_so_far_accuracy:
                    best_so_far_accuracy = accuracy
                    remove_total = j
                if accuracy > best_subset_accuracy:
                    best_subset_accuracy = accuracy
                    remove_subset = j
                    
        if remove_total != -1:
            best_subset.remove(remove_total)
            best_total.remove(remove_total)
            print("\n\nFeature set ", best_subset, " was best, accuracy is ", best_so_far_accuracy, "%\n\n")
        else:
            print("\n\n(Warning, Accuracy has decreases! Continuing search in case of local maxima)")
            best_subset.remove(remove_subset)
            print("Feature set ", best_subset, " was best, accuracy is ", best_subset_accuracy, "%\n\n")
    #Finished looping all numbers of features
    print("Finished search!! The best feature subset is ", best_total, " which has an accuracy of ", best_so_far_accuracy, "%")
    

    
                    
                

In [26]:
def leave_one_out_cross_validation(data, cur_subset, num_instances):
    num_corr_classfied = 0
    for i in range(num_instances):
        nearest_neighbor_location = float("inf")
        nearest_neighbor_distance = float("inf")
        for j in range(num_instances):
            if j == i:
                continue
            else:
                distance = 0
                for k in range(len(cur_subset)):
                    distance = distance + pow((data[j][cur_subset[k]] - data[i][cur_subset[k]]), 2)
                distance = math.sqrt(distance)
                
                if distance < nearest_neighbor_distance:
                    nearest_neighbor_distance = distance
                    nearest_neighbor_location = j
        
        neighbor = nearest_neighbor_location
        
        if data[neighbor][0] == data[i][0]:
            num_corr_classfied += 1
    
    accuracy = (num_corr_classfied/num_instances) *100
    return accuracy
        
    
    
    

In [33]:
def run():
    print("Welcome to Xinning Dong's Feature Selection Algorithm.")
    file = input("Type in the name of the file to test: ")
    data = open(file, 'r')
    
    readline = data.readline()
    num_features = len(readline.split()) - 1
    
    data.seek(0)
    num_instances = sum(1 for i in data)
    data.seek(0)
    
    #load data as a 2D array
    instances = [[] for i in range(num_instances)]
    for i in range(num_instances):
        instances[i] = [float(j) for j in data.readline().split()]
        
    
    print("Type the number of the algorithm you want to run.")
    print("1) Forward Selection")
    print("2) Backward Elimination")
    switch = int(input())
    print("This data set has ", num_features, " features (not including the class attribute), with ", num_instances,
          " instances." )
        
    
    #normalize data by deducting average of the data and then devide by the standard deviation of the data
    avg = []
    for i in range(1, num_features + 1):
        avg.append((sum(row[i] for row in instances)) / num_instances)
    
    std = []
    for i in range(1, num_features + 1):
        std.append(math.sqrt(sum(pow((row[i] - avg[i-1]), 2) for row in instances) / num_instances))
    
    for i in range(num_instances):
        for j in range(1, num_features + 1):
            instances[i][j] = (instances[i][j] - avg[j-1])/std[j-1]
    
    #run with all features
    all_subset = []
    for i in range(1, num_features + 1):
        all_subset.append(i)
    accuracy = leave_one_out_cross_validation(instances, all_subset, num_instances)
    print("Running nearest neighbor with all ", num_features, " features, using \"leaving-one-out\" evaluation, 
          I get an accuracy of ", accuracy, "%")    
    
    print("Beginning search.\n\n")
    
    if switch == 1:
        forward_selection(instances, num_features, num_instances)
    elif switch == 2:
        backward_elimination(instances, num_features, num_instances)
    

def timed_run():
    start = time.time()
    run()
    end = time.time()
    print("Function finishes in ", end-start, "seconds.")
    